In [2]:
import numpy as np
import pandas as pd
import random
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from stable_baselines3.common.vec_env import DummyVecEnv

from env import StockLearningEnv
import config

from time import time

# PPO 训练

In [3]:
df = pd.read_csv('./data_file/train.csv')
e_train_gym = StockLearningEnv(df = df, **config.ENV_TRAIN_PARAMS)

加载数据缓存
数据缓存成功!


In [40]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Beta,Normal
import math


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('devic:', device)

class BetaActor(nn.Module):
	def __init__(self, state_dim, action_dim, net_width):
		super(BetaActor, self).__init__()

		self.l1 = nn.Linear(state_dim, net_width)
		self.l2 = nn.Linear(net_width, net_width)
		self.alpha_head = nn.Linear(net_width, action_dim)
		self.beta_head = nn.Linear(net_width, action_dim)
		self.attention1 = nn.MultiheadAttention(net_width, 1, batch_first=True)
		self.attention2 = nn.MultiheadAttention(net_width, 1, batch_first=True)

	def forward(self, state):
		a = torch.tanh(self.l1(state))
		a = torch.tanh(self.l2(a))

		alpha = F.softplus(self.alpha_head(a)) + 1.0
		beta = F.softplus(self.beta_head(a)) + 1.0

		return alpha,beta

	def get_dist(self,state):
		alpha,beta = self.forward(state)
		dist = Beta(alpha, beta)
		return dist

	def dist_mode(self,state):
		alpha, beta = self.forward(state)
		mode = (alpha) / (alpha + beta)
		return mode

class GaussianActor_musigma(nn.Module):
	def __init__(self, state_dim, action_dim, net_width):
		super(GaussianActor_musigma, self).__init__()

		self.l1 = nn.Linear(state_dim, net_width)
		self.l2 = nn.Linear(net_width, net_width)
		self.mu_head = nn.Linear(net_width, action_dim)
		self.sigma_head = nn.Linear(net_width, action_dim)

	def forward(self, state):
		a = torch.tanh(self.l1(state))
		a = torch.tanh(self.l2(a))
		mu = torch.sigmoid(self.mu_head(a))
		sigma = F.softplus( self.sigma_head(a) )
		return mu,sigma

	def get_dist(self, state):
		mu,sigma = self.forward(state)
		dist = Normal(mu,sigma)
		return dist

class GaussianActor_mu(nn.Module):
	def __init__(self, state_dim, action_dim, net_width, log_std=0):
		super(GaussianActor_mu, self).__init__()

		self.l1 = nn.Linear(state_dim, net_width)
		self.l2 = nn.Linear(net_width, net_width)
		self.mu_head = nn.Linear(net_width, action_dim)
		self.mu_head.weight.data.mul_(0.1)
		self.mu_head.bias.data.mul_(0.0)

		self.action_log_std = nn.Parameter(torch.ones(1, action_dim) * log_std)

	def forward(self, state):
		a = torch.relu(self.l1(state))
		a = torch.relu(self.l2(a))
		mu = torch.sigmoid(self.mu_head(a))
		return mu

	def get_dist(self,state):
		mu = self.forward(state)
		action_log_std = self.action_log_std.expand_as(mu)
		action_std = torch.exp(action_log_std)

		dist = Normal(mu, action_std)
		return dist


class Critic(nn.Module):
	def __init__(self, state_dim,net_width):
		super(Critic, self).__init__()
		self.net_width = net_width
		
		self.C1 = nn.Linear(state_dim, net_width)
		self.C2 = nn.Linear(net_width, net_width)
		self.C3 = nn.Linear(net_width, 1)

		self.attention1 = nn.MultiheadAttention(net_width, 1, batch_first=True)
		self.attention2 = nn.MultiheadAttention(net_width, 1, batch_first=True)

	def forward(self, state):
		v = torch.tanh(self.C1(state))

		v = v.view(-1, 1, self.net_width)

		v, attn_output_weights = self.attention1(v, v, v)

		v = v.view(-1, self.net_width)


		v = torch.tanh(self.C2(v))
		v = v.view(-1, 1, self.net_width)
		v, attn_output_weights = self.attention2(v, v, v)
		v = v.view(-1, self.net_width)

		v = self.C3(v)
		return v



class Attention_PPO(object):
	def __init__(
		self,
		state_dim,
		action_dim,
		env_with_Dead,
		gamma=0.99,
		lambd=0.95,
		clip_rate=0.2,
		K_epochs=10,
		net_width=256,
		a_lr=3e-4,
		c_lr=3e-4,
		l2_reg = 1e-3,
		dist='Beta',
		a_optim_batch_size = 64,
		c_optim_batch_size = 64,
		entropy_coef = 0,
		entropy_coef_decay = 0.9998
	):
		if dist == 'Beta':
			self.actor = BetaActor(state_dim, action_dim, net_width).to(device)
		elif dist == 'GS_ms':
			self.actor = GaussianActor_musigma(state_dim, action_dim, net_width).to(device)
		elif dist == 'GS_m':
			self.actor = GaussianActor_mu(state_dim, action_dim, net_width).to(device)
		else: print('Dist Error')
		self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=a_lr)
		self.dist = dist

		self.critic = Critic(state_dim, net_width).to(device)
		self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=c_lr)

		self.env_with_Dead = env_with_Dead
		self.action_dim = action_dim
		self.clip_rate = clip_rate
		self.gamma = gamma
		self.lambd = lambd
		self.clip_rate = clip_rate
		self.K_epochs = K_epochs
		self.data = []
		self.l2_reg = l2_reg
		self.a_optim_batch_size = a_optim_batch_size
		self.c_optim_batch_size = c_optim_batch_size
		self.entropy_coef = entropy_coef
		self.entropy_coef_decay = entropy_coef_decay

	def select_action(self, state):#only used when interact with the env
		with torch.no_grad():
			state = torch.FloatTensor(state.reshape(1, -1)).to(device)
			dist = self.actor.get_dist(state)
			a = dist.sample()
			a = torch.clamp(a, 0, 1)
			logprob_a = dist.log_prob(a).cpu().numpy().flatten()
			return a.cpu().numpy().flatten(), logprob_a

	def evaluate(self, state):#only used when evaluate the policy.Making the performance more stable
		with torch.no_grad():
			state = torch.FloatTensor(state.reshape(1, -1)).to(device)
			if self.dist == 'Beta':
				a = self.actor.dist_mode(state)
			if self.dist == 'GS_ms':
				a,b = self.actor(state)
			if self.dist == 'GS_m':
				a = self.actor(state)
			return a.cpu().numpy().flatten(),0.0


	def train(self):
		self.entropy_coef*=self.entropy_coef_decay
		s, a, r, s_prime, logprob_a, done_mask, dw_mask = self.make_batch()


		''' Use TD+GAE+LongTrajectory to compute Advantage and TD target'''
		with torch.no_grad():
			vs = self.critic(s)
			vs_ = self.critic(s_prime)

			'''dw for TD_target and Adv'''
			deltas = r + self.gamma * vs_ * (1 - dw_mask) - vs

			deltas = deltas.cpu().flatten().numpy()
			adv = [0]

			'''done for GAE'''
			for dlt, mask in zip(deltas[::-1], done_mask.cpu().flatten().numpy()[::-1]):
				advantage = dlt + self.gamma * self.lambd * adv[-1] * (1 - mask)
				adv.append(advantage)
			adv.reverse()
			adv = copy.deepcopy(adv[0:-1])
			adv = torch.tensor(adv).unsqueeze(1).float().to(device)
			td_target = adv + vs
			adv = (adv - adv.mean()) / ((adv.std()+1e-4))  #sometimes helps


		"""Slice long trajectopy into short trajectory and perform mini-batch PPO update"""
		a_optim_iter_num = int(math.ceil(s.shape[0] / self.a_optim_batch_size))
		c_optim_iter_num = int(math.ceil(s.shape[0] / self.c_optim_batch_size))
		for i in range(self.K_epochs):

			#Shuffle the trajectory, Good for training
			perm = np.arange(s.shape[0])
			np.random.shuffle(perm)
			perm = torch.LongTensor(perm).to(device)
			s, a, td_target, adv, logprob_a = \
				s[perm].clone(), a[perm].clone(), td_target[perm].clone(), adv[perm].clone(), logprob_a[perm].clone()

			'''update the actor'''
			for i in range(a_optim_iter_num):
				index = slice(i * self.a_optim_batch_size, min((i + 1) * self.a_optim_batch_size, s.shape[0]))
				distribution = self.actor.get_dist(s[index])
				dist_entropy = distribution.entropy().sum(1, keepdim=True)
				logprob_a_now = distribution.log_prob(a[index])
				ratio = torch.exp(logprob_a_now.sum(1,keepdim=True) - logprob_a[index].sum(1,keepdim=True))  # a/b == exp(log(a)-log(b))

				surr1 = ratio * adv[index]
				surr2 = torch.clamp(ratio, 1 - self.clip_rate, 1 + self.clip_rate) * adv[index]
				a_loss = -torch.min(surr1, surr2) - self.entropy_coef * dist_entropy

				self.actor_optimizer.zero_grad()
				a_loss.mean().backward()
				torch.nn.utils.clip_grad_norm_(self.actor.parameters(), 40)
				self.actor_optimizer.step()

			'''update the critic'''
			for i in range(c_optim_iter_num):
				index = slice(i * self.c_optim_batch_size, min((i + 1) * self.c_optim_batch_size, s.shape[0]))
				c_loss = (self.critic(s[index]) - td_target[index]).pow(2).mean()
				for name,param in self.critic.named_parameters():
					if 'weight' in name:
						c_loss += param.pow(2).sum() * self.l2_reg

				self.critic_optimizer.zero_grad()
				c_loss.backward()
				self.critic_optimizer.step()


	def make_batch(self):
		s_lst, a_lst, r_lst, s_prime_lst, logprob_a_lst, done_lst, dw_lst = [], [], [], [], [], [], []
		for transition in self.data:
			s, a, r, s_prime, logprob_a, done, dw = transition

			s_lst.append(s)
			a_lst.append(a)
			logprob_a_lst.append(logprob_a)
			r_lst.append([r])
			s_prime_lst.append(s_prime)
			done_lst.append([done])
			dw_lst.append([dw])

		if not self.env_with_Dead:
			'''Important!!!'''
			# env_without_DeadAndWin: deltas = r + self.gamma * vs_ - vs
			# env_with_DeadAndWin: deltas = r + self.gamma * vs_ * (1 - dw) - vs
			dw_lst = (np.array(dw_lst)*False).tolist()

		self.data = [] #Clean history trajectory

		'''list to tensor'''
		with torch.no_grad():
			s, a, r, s_prime, logprob_a, done_mask, dw_mask = \
				torch.tensor(s_lst, dtype=torch.float).to(device), \
				torch.tensor(a_lst, dtype=torch.float).to(device), \
				torch.tensor(r_lst, dtype=torch.float).to(device), \
				torch.tensor(s_prime_lst, dtype=torch.float).to(device), \
				torch.tensor(logprob_a_lst, dtype=torch.float).to(device), \
				torch.tensor(done_lst, dtype=torch.float).to(device), \
				torch.tensor(dw_lst, dtype=torch.float).to(device),


		return s, a, r, s_prime, logprob_a, done_mask, dw_mask


	def put_data(self, transition):
		self.data.append(transition)

	def save(self,episode):
		torch.save(self.critic.state_dict(), "./attention_ppo/ppo_critic{}.pth".format(episode))
		torch.save(self.actor.state_dict(), "./attention_ppo/ppo_actor{}.pth".format(episode))


	def load(self,episode):
		self.critic.load_state_dict(torch.load("./attention_ppo/ppo_critic{}.pth".format(episode)))
		self.actor.load_state_dict(torch.load("./attention_ppo/ppo_actor{}.pth".format(episode)))

devic: cuda


In [44]:
# from PPO import PPO
# from Attention_PPO import Attention_PPO
kwargs = {
    "state_dim": 801,
    "action_dim": 50,
    "env_with_Dead":True
}
model = Attention_PPO(**kwargs)

In [45]:
T_horizon = 100
traj_lenth = 0
total_steps = 0
Max_train_steps = 50000
save_interval = 10000

In [46]:
model.load(100)

In [47]:
while total_steps < Max_train_steps:
    s, done, steps, ep_r = e_train_gym.reset(), False, 0, 0

    '''Interact & trian'''
    while not done:
        traj_lenth += 1
        # print('s:', s)
        s = np.array(s)
        # print(np.shape(s))
        a, logprob_a = model.select_action(s)

        
        s_prime, r, done, info = e_train_gym.step(a)
       

        '''distinguish done between dead|win(dw) and reach env._max_episode_steps(rmax); done = dead|win|rmax'''
        '''dw for TD_target and Adv; done for GAE'''
        if done :
            dw = True
        else:
            dw = False

        model.put_data((s, a, r, s_prime, logprob_a, done, dw))
        s = s_prime
        ep_r += r

        '''update if its time'''
        
        if traj_lenth % T_horizon == 0:
            model.train()
            traj_lenth = 0

        total_steps += 1
        
        '''save model'''
        if total_steps % save_interval==0:
            model.save(total_steps)

KeyboardInterrupt: 